In [1]:
import pandas as pd
import numpy as np
from prettytable import PrettyTable

In [2]:
app_files = ["console_2018-12-27_12-16-52.log", "console_2018-12-27_12-45-57.log", 
         "console_2018-12-27_13-49-15.log", "console_2018-12-27_14-34-15.log"]
rl_files = ["run_length_2018-12-27_12-16-18.log", "run_length_2018-12-27_12-53-01.log", 
            "run_length_2018-12-27_13-49-32.log", "run_length_2018-12-27_14-34-55.log"]

In [3]:
objects = {"[0, 0, 0, 0, 0, 0]":"none",
           "[1, 0, 0, 0, 0, 0]":"cards", 
           "[0, 1, 0, 0, 0, 0]":"dice", 
           "[0, 0, 1, 0, 0, 0]":"key",
           "[0, 0, 0, 1, 0, 0]":"map", 
           "[0, 0, 0, 0, 1, 0]":"phone", 
           "[0, 0, 0, 0, 0, 1]":"ball"}

In [4]:
def read_app_file(file_name):
    with open("data/" + file_name, "r") as f:
        lines  = f.readlines()
        data = lines[2:-2]
        data_list = []
        for line in data:
            object_time = line.split("$1")[1].split("-")[1].strip().split(",")
            if object_time[0] == 'cancel':
                data_list.pop()
            else:
                object_time[1] = float(object_time[1].strip())
                data_list.append(object_time)
    return data_list

In [5]:
def read_rl_file(file_name):
    with open("data/" + file_name, "r") as f:
        lines  = f.readlines()
        hit_scan_list = []
        for line in lines:
            parts = line.split(";")
            time = float(parts[1].split("-")[1])
            glass1 = parts[2].split("-")[1]
            glass2 = parts[3].split("-")[1]
            hit_scan_list.append((time, objects.get(glass1), objects.get(glass2)))
    return hit_scan_list

In [6]:
def create_ds(app_file, rl_file):
    app_data = read_app_file(app_file)
    hsl_data = read_rl_file(rl_file)        

    ot_rl = []
    last_e_ind=0
    for i, ot in enumerate(app_data):
        s_time = ot[1]

        if i != 0:
            s_ind = last_e_ind
        else:
            for j, hsl in enumerate(hsl_data):
                if hsl[0] > s_time:
                    s_ind = j-1
                    break

        if i != (len(app_data) -1):
            e_time = app_data[i+1][1]
            e_ind = 0
            for j, hsl in enumerate(hsl_data[s_ind:]):
                if hsl[0] > e_time:
                    e_ind = s_ind + j
                    break
            rl = hsl_data[s_ind:e_ind]
            last_e_ind = e_ind
        else:
            rl = hsl_data[s_ind:]
        
        new_ot = ot.copy()
        new_ot.append(rl)
        ot_rl.append(new_ot)
    return ot_rl   

In [7]:
def process_ds(ot_rl_list):
    new_ot_rl = []
    for ot_rl in ot_rl_list:
        obj = ot_rl[0]
        rl_data = ot_rl[2]
        timeseries = []
        detections = []
        for rl in rl_data:
            timeseries.append(rl[0])
            detection = 0
            if obj == rl[1] and obj == rl[2]:
                detection = 1
            detections.append(detection)
        rld = {"ts":timeseries, "hits":detections}
        new_data = (obj, ot_rl[1], rld)
        new_ot_rl.append(new_data)
    return new_ot_rl

In [8]:
tot_ot_rl = []
for i in range(len(app_files)):
    ot_rl = create_ds(app_files[i], rl_files[i])
    tot_ot_rl.extend(ot_rl)

In [9]:
new_ot_rl = process_ds(tot_ot_rl)

In [10]:
class RunLengthFilter:
    
    def __init__(self, ot_rl_list, true_look_size, true_look_delta):
        self.data = ot_rl_list
        self.true_size = true_look_size
        self.true_delta = true_look_delta
        self.true_looks = self.determine_true_looks()
        
    def rl_filter(self, window_size, num_ones):
        
        df = {'object':[], 'app_time':[], 'look_time':[], 'true_look_start':self.true_looks, 'is_true_look':[]}
        diff = window_size - num_ones
        
        for idx, ot_rl in enumerate(self.data):
            df['object'].append(ot_rl[0])
            df['app_time'].append(ot_rl[1])
            
            rl_data = ot_rl[2]
            ts = rl_data['ts']
            hits = rl_data['hits']
            
            s_time = look_time = 0
            true_look = self.true_looks[idx]
            
            for i in range(-diff, len(ts)):
                if i<0:
                    s_idx = 0
                else:
                    s_idx = i
                e_idx = i + window_size
                if sum(hits[s_idx:e_idx]) == num_ones:
                    look_time = ts[e_idx-1]
                    s_time = ts[s_idx]
                    break
            df['look_time'].append(look_time)
            if s_time == true_look:
                df["is_true_look"].append(1)
            else:
                df["is_true_look"].append(0)
                
        df = pd.DataFrame(df)
        df['latency'] = df['look_time'] - df['app_time']
        return df
    
    def determine_true_looks(self):
        start_look = []
        for ot_rl in self.data:
            rl_data = ot_rl[2]
            ts = rl_data['ts']
            hits = rl_data['hits']
            start_time = 0
            delta = self.true_delta
            start_idx = 0
            for i in range(len(ts)):
                start_idx = i
                e_idx = i + self.true_size
                if sum(hits[i:e_idx]) == self.true_size:
                    while (delta > 0):
                        if i-delta >= 0 and hits[i-delta] == 1:
                            start_idx = i-delta
                            break
                        delta -= 1
                    break
            start_look.append(ts[start_idx])
        return start_look

In [11]:
rl = RunLengthFilter(new_ot_rl, 4, 3)

In [12]:
rl3_df = rl.rl_filter(3, 3)
rl4_df = rl.rl_filter(4, 4)
rl1_df = rl.rl_filter(1, 1)

In [13]:
def summary(df_rl):
    rl_dict = {}
    for obj in df_rl["object"].unique():
        df = df_rl[df_rl["object"] == obj]
        m = round(np.mean(df["latency"]), 3)
        s = round(np.std(df["latency"]), 3)
        tpr = get_tpr(df)
        rl_dict[obj] = [m, s, tpr]
    return rl_dict

def get_tpr(df):
    return round((sum(df['is_true_look']) * 100) / len(df), 3)

In [14]:
rl3_dict = summary(rl3_df)
rl4_dict = summary(rl4_df)
rl1_dict = summary(rl1_df)

In [15]:
table = PrettyTable(["Object", "RL3 Mean(Std)", "RL3 TPR", "RL4 Mean(Std)", "RL4 TPR", "RL1 Mean(Std)", "RL1 TPR"])
for obj in rl3_df["object"].unique():
    table.add_row([obj, "{} ({})".format(rl3_dict[obj][0], rl3_dict[obj][1]), rl3_dict[obj][2], 
                   "{} ({})".format(rl4_dict[obj][0], rl4_dict[obj][1]), rl4_dict[obj][2],
                   "{} ({})".format(rl1_dict[obj][0], rl1_dict[obj][1]), rl1_dict[obj][2]])
print(table)

+--------+--------------------+---------+--------------------+---------+--------------------+---------+
| Object |   RL3 Mean(Std)    | RL3 TPR |   RL4 Mean(Std)    | RL4 TPR |   RL1 Mean(Std)    | RL1 TPR |
+--------+--------------------+---------+--------------------+---------+--------------------+---------+
|  ball  | 1079.949 (277.459) |  100.0  | 1082.532 (277.079) |  100.0  | 1072.557 (278.293) |  100.0  |
| cards  | 1171.658 (299.386) |  100.0  | 1173.544 (299.258) |  100.0  | 1165.43 (299.336)  |  100.0  |
|  dice  | 833.291 (213.675)  |  98.734 | 838.291 (214.913)  |  100.0  | 826.405 (213.138)  |  98.734 |
|  key   |  896.282 (180.29)  |  100.0  | 899.064 (180.718)  |  100.0  | 890.115 (180.195)  |  100.0  |
|  map   | 841.949 (192.959)  |  100.0  | 845.154 (193.077)  |  100.0  | 834.782 (193.257)  |  100.0  |
+--------+--------------------+---------+--------------------+---------+--------------------+---------+
